## 常见的CNN网络结构及关系

**AlexNet** - > VGG: VGG可以看成是加深版本的AlexNet. 都是conv layer + FC layer.

**Network in Network** -> GoogLeNet: NIN本身大家可能不太熟悉，但是我个人觉得是蛮不错的工作，Lin Min挺厉害。GoogLeNet这篇论文里面也对NIN大为赞赏。NIN利用Global average pooling去掉了FC layer, 大大减少了模型大小，本身的网络套网络的结构，也激发了后来的GoogLeNet里面的各种sub-network和inception结构的设计. 

**ResNet**：这个网络跟前面几个网络都不同。我清楚记得这篇论文是在去年年底我去开NIPS的时候release到arxiv上的。当时我开会间歇中看着论文里面在cifar上面的一千层的resnet都目瞪狗呆了。。。然后再看到ResNet刷出了imagenet和COCO各个比赛的冠军，当时就觉得如果这论文是投CVPR, 那是绝对没有争议的Best paper, 果不其然。好像resnet后来又有些争议，说resnet跟highway network很像啥的，或者跟RNN结构类似，但都不可动摇ResNet对Computer Vision的里程碑贡献。当然，训练这些网络，还有些非常重要的trick, 如dropout, batch normalization等也功不可没。等我有时间了可以再写写这些tricks。

总的来说就是三个方向

1. LeNet, AlexNet, VGG.
2. GoogLeNet, Google Inception.
3. ResNet.


- **LeNet**. The first successful applications of Convolutional Networks were developed by Yann LeCun in 1990’s. Of these, the best known is the LeNet architecture that was used to read zip codes, digits, etc.
- AlexNet. The first work that popularized Convolutional Networks in Computer Vision was the AlexNet, developed by Alex Krizhevsky, Ilya Sutskever and Geoff Hinton. The AlexNet was submitted to the ImageNet ILSVRC challenge in 2012 and significantly outperformed the second runner-up (top 5 error of 16% compared to runner-up with 26% error). The Network had a very similar architecture to LeNet, but was deeper, bigger, and featured Convolutional Layers stacked on top of each other (previously it was common to only have a single CONV layer always immediately followed by a POOL layer).
- **ZF Net**. The ILSVRC 2013 winner was a Convolutional Network from Matthew Zeiler and Rob Fergus. It became known as the ZFNet (short for Zeiler & Fergus Net). It was an improvement on AlexNet by tweaking the architecture hyperparameters, in particular by expanding the size of the middle convolutional layers and making the stride and filter size on the first layer smaller.
- **GoogLeNet**. The ILSVRC 2014 winner was a Convolutional Network from Szegedy et al. from Google. Its main contribution was the development of an Inception Module that dramatically reduced the number of parameters in the network (4M, compared to AlexNet with 60M). Additionally, this paper uses Average Pooling instead of Fully Connected layers at the top of the ConvNet, eliminating a large amount of parameters that do not seem to matter much. There are also several followup versions to the GoogLeNet, most recently Inception-v4.
- **VGGNet**. The runner-up in ILSVRC 2014 was the network from Karen Simonyan and Andrew Zisserman that became known as the VGGNet. Its main contribution was in showing that the depth of the network is a critical component for good performance. Their final best network contains 16 CONV/FC layers and, appealingly, features an extremely homogeneous architecture that only performs 3x3 convolutions and 2x2 pooling from the beginning to the end. Their pretrained model is available for plug and play use in Caffe. A downside of the VGGNet is that it is more expensive to evaluate and uses a lot more memory and parameters (140M). Most of these parameters are in the first fully connected layer, and it was since found that these FC layers can be removed with no performance downgrade, significantly reducing the number of necessary parameters.
- **ResNet**. Residual Network developed by Kaiming He et al. was the winner of ILSVRC 2015. It features special skip connections and a heavy use of batch normalization. The architecture is also missing fully connected layers at the end of the network. The reader is also referred to Kaiming’s presentation (video, slides), and some recent experiments that reproduce these networks in Torch. ResNets are currently by far state of the art Convolutional Neural Network models and are the default choice for using ConvNets in practice (as of May 10, 2016). In particular, also see more recent developments that tweak the original architecture from Kaiming He et al. Identity Mappings in Deep Residual Networks (published March 2016).

### AlexNet

[AlexNet](http://vision.stanford.edu/teaching/cs231b_spring1415/slides/alexnet_tugce_kyunghee.pdf)是2012年最先出来的CNN, 其在ILSVRC比赛中获得了16.4%的成功率, 确立了神经网络在计算机视觉中的地位. 其主要在LeNet的基础上, 利用了很多Trick使得网络得以极大加深.  其结构如下

<img src="https:\/\/ooo.0o0.ooo\/2017\/04\/24\/58fd9c88bfafd.png" width=400></img>
<img src="https:\/\/ooo.0o0.ooo\/2017\/04\/24\/58fd9c89eb840.png" width=500></img>

1. 使用Relu作为CNN的激活函数, Relu的好处是1. 相比引入Sigmoid稀疏性, 2.解决了Sigmoid的梯度弥散问题, 这是由于Sigmoid的输出只有0-1.
2. 使用Dropout, 其好处是相当于生成了很多随机样本, 避免了过拟合.
3. 使用重叠的最大池化.
4. 使用了LRN(这个现在来看没有太多的用处, 就是对Relu输出的结果进行归一化), 使得大的神经元更大, 小的神经元更小.
5. 数据增强, 对原来的数据集进行变形, 扩大数据集.

其结构是8层, 其中前面5层是CNN, 后面3层是FN, 最后一层是一个SOFTMAX输出.

常用的避免过拟合Trick, DAAR

1. Dropout.
2. Adam(随机梯度下降以及其变形来进行训练, 比如动量)
3. 数据增强.(Data Augmentation)
4. 使用修正线性单元作为激活函数.
5. Weight Dacay(正则化的系数), DL里一般放在SGD里.

为什么要用Softmax而不用max.

1. 想让输出对概率的影响是乘性的.
2. 反向传播计算方便.



In [1]:
from datetime import datetime
import math
import time
import tensorflow as tf


batch_size=32
num_batches=100

def print_activations(t):
    print(t.op.name, ' ', t.get_shape().as_list())


def inference(images):
    parameters = []
    # conv1
    with tf.name_scope('conv1') as scope:
        kernel = tf.Variable(tf.truncated_normal([11, 11, 3, 64], dtype=tf.float32,
                                                 stddev=1e-1), name='weights')
        conv = tf.nn.conv2d(images, kernel, [1, 4, 4, 1], padding='SAME')
        biases = tf.Variable(tf.constant(0.0, shape=[64], dtype=tf.float32),
                             trainable=True, name='biases')
        bias = tf.nn.bias_add(conv, biases)
        conv1 = tf.nn.relu(bias, name=scope)
        print_activations(conv1)
        parameters += [kernel, biases]


  # pool1
    lrn1 = tf.nn.lrn(conv1, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75, name='lrn1')
    pool1 = tf.nn.max_pool(lrn1,
                           ksize=[1, 3, 3, 1],
                           strides=[1, 2, 2, 1],
                           padding='VALID',
                           name='pool1')
    print_activations(pool1)

  # conv2
    with tf.name_scope('conv2') as scope:
        kernel = tf.Variable(tf.truncated_normal([5, 5, 64, 192], dtype=tf.float32,
                                                 stddev=1e-1), name='weights')
        conv = tf.nn.conv2d(pool1, kernel, [1, 1, 1, 1], padding='SAME')
        biases = tf.Variable(tf.constant(0.0, shape=[192], dtype=tf.float32),
                             trainable=True, name='biases')
        bias = tf.nn.bias_add(conv, biases)
        conv2 = tf.nn.relu(bias, name=scope)
        parameters += [kernel, biases]
    print_activations(conv2)

  # pool2
    lrn2 = tf.nn.lrn(conv2, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75, name='lrn2')
    pool2 = tf.nn.max_pool(lrn2,
                           ksize=[1, 3, 3, 1],
                           strides=[1, 2, 2, 1],
                           padding='VALID',
                           name='pool2')
    print_activations(pool2)

  # conv3
    with tf.name_scope('conv3') as scope:
        kernel = tf.Variable(tf.truncated_normal([3, 3, 192, 384],
                                                 dtype=tf.float32,
                                                 stddev=1e-1), name='weights')
        conv = tf.nn.conv2d(pool2, kernel, [1, 1, 1, 1], padding='SAME')
        biases = tf.Variable(tf.constant(0.0, shape=[384], dtype=tf.float32),
                             trainable=True, name='biases')
        bias = tf.nn.bias_add(conv, biases)
        conv3 = tf.nn.relu(bias, name=scope)
        parameters += [kernel, biases]
        print_activations(conv3)

  # conv4
    with tf.name_scope('conv4') as scope:
        kernel = tf.Variable(tf.truncated_normal([3, 3, 384, 256],
                                                 dtype=tf.float32,
                                                 stddev=1e-1), name='weights')
        conv = tf.nn.conv2d(conv3, kernel, [1, 1, 1, 1], padding='SAME')
        biases = tf.Variable(tf.constant(0.0, shape=[256], dtype=tf.float32),
                             trainable=True, name='biases')
        bias = tf.nn.bias_add(conv, biases)
        conv4 = tf.nn.relu(bias, name=scope)
        parameters += [kernel, biases]
        print_activations(conv4)

  # conv5
    with tf.name_scope('conv5') as scope:
        kernel = tf.Variable(tf.truncated_normal([3, 3, 256, 256],
                                                 dtype=tf.float32,
                                                 stddev=1e-1), name='weights')
        conv = tf.nn.conv2d(conv4, kernel, [1, 1, 1, 1], padding='SAME')
        biases = tf.Variable(tf.constant(0.0, shape=[256], dtype=tf.float32),
                             trainable=True, name='biases')
        bias = tf.nn.bias_add(conv, biases)
        conv5 = tf.nn.relu(bias, name=scope)
        parameters += [kernel, biases]
        print_activations(conv5)

  # pool5
    pool5 = tf.nn.max_pool(conv5,
                           ksize=[1, 3, 3, 1],
                           strides=[1, 2, 2, 1],
                           padding='VALID',
                           name='pool5')
    print_activations(pool5)

    return pool5, parameters


def time_tensorflow_run(session, target, info_string):
#  """Run the computation to obtain the target tensor and print timing stats.
#
#  Args:
#    session: the TensorFlow session to run the computation under.
#    target: the target Tensor that is passed to the session's run() function.
#    info_string: a string summarizing this run, to be printed with the stats.
#
#  Returns:
#    None
#  """
    num_steps_burn_in = 10
    total_duration = 0.0
    total_duration_squared = 0.0
    for i in range(num_batches + num_steps_burn_in):
        start_time = time.time()
        _ = session.run(target)
        duration = time.time() - start_time
        if i >= num_steps_burn_in:
            if not i % 10:
                print ('%s: step %d, duration = %.3f' %
                       (datetime.now(), i - num_steps_burn_in, duration))
            total_duration += duration
            total_duration_squared += duration * duration
    mn = total_duration / num_batches
    vr = total_duration_squared / num_batches - mn * mn
    sd = math.sqrt(vr)
    print ('%s: %s across %d steps, %.3f +/- %.3f sec / batch' %
           (datetime.now(), info_string, num_batches, mn, sd))



def run_benchmark():
#  """Run the benchmark on AlexNet."""
    with tf.Graph().as_default():
    # Generate some dummy images.
        image_size = 224
    # Note that our padding definition is slightly different the cuda-convnet.
    # In order to force the model to start with the same activations sizes,
    # we add 3 to the image_size and employ VALID padding above.
        images = tf.Variable(tf.random_normal([batch_size,
                                           image_size,
                                           image_size, 3],
                                          dtype=tf.float32,
                                          stddev=1e-1))

    # Build a Graph that computes the logits predictions from the
    # inference model.
        pool5, parameters = inference(images)

    # Build an initialization operation.
        init = tf.global_variables_initializer()

    # Start running operations on the Graph.
        config = tf.ConfigProto()
        config.gpu_options.allocator_type = 'BFC'
        sess = tf.Session(config=config)
        sess.run(init)

    # Run the forward benchmark.
        time_tensorflow_run(sess, pool5, "Forward")

    # Add a simple objective so we can calculate the backward pass.
        objective = tf.nn.l2_loss(pool5)
    # Compute the gradient with respect to all the parameters.
        grad = tf.gradients(objective, parameters)
    # Run the backward benchmark.
        time_tensorflow_run(sess, grad, "Forward-backward")


run_benchmark()


(u'conv1', ' ', [32, 56, 56, 64])
(u'pool1', ' ', [32, 27, 27, 64])
(u'conv2', ' ', [32, 27, 27, 192])
(u'pool2', ' ', [32, 13, 13, 192])
(u'conv3', ' ', [32, 13, 13, 384])
(u'conv4', ' ', [32, 13, 13, 256])
(u'conv5', ' ', [32, 13, 13, 256])
(u'pool5', ' ', [32, 6, 6, 256])
2017-04-24 14:44:35.388179: step 0, duration = 1.549
2017-04-24 14:44:49.972340: step 10, duration = 1.519
2017-04-24 14:45:05.181623: step 20, duration = 1.303
2017-04-24 14:45:20.558017: step 30, duration = 1.721
2017-04-24 14:45:34.739758: step 40, duration = 1.580
2017-04-24 14:45:49.486042: step 50, duration = 1.632
2017-04-24 14:46:05.115791: step 60, duration = 1.387
2017-04-24 14:46:18.969777: step 70, duration = 1.573
2017-04-24 14:46:32.659390: step 80, duration = 1.350
2017-04-24 14:46:46.792801: step 90, duration = 1.435
2017-04-24 14:46:59.440975: Forward across 100 steps, 1.456 +/- 0.146 sec / batch
2017-04-24 14:47:46.509361: step 0, duration = 3.815
2017-04-24 14:48:27.505089: step 10, duration = 4.